In [18]:
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual 
import ipywidgets as widgets
import numpy as np
import math
import plotly.express as px

In [19]:
def show_analysis(file_name):
    fig=plt.figure(figsize=(16,5), dpi=200, facecolor='w', edgecolor='k')
    fig.tight_layout()
    plt.subplot(1,3,1)
    df = pd.read_csv(file_name)
    plt.gca().set_aspect('equal')
    plt.plot(df.loc_x.to_numpy(),df.loc_y.to_numpy())

    
    plt.subplot(2,3,2)
    plt.tight_layout()
    plt.title("velocity")
    plt.gca().set_aspect('auto')
    plt.plot(df.t.to_numpy(), df.v_x.to_numpy(), ".:", label="x")
    plt.plot(df.t.to_numpy(), df.v_y.to_numpy(), ".:", label="y")
    plt.xlabel("t")
    plt.legend()
    
    plt.subplot(2,3,3)
    plt.tight_layout()
    plt.title("acceleration")
    plt.gca().set_aspect('auto')
    plt.plot(df.t.to_numpy(), df.a_x.to_numpy(), ".:", label="x")
    plt.plot(df.t.to_numpy(), df.a_y.to_numpy(), ".:", label="y")
    plt.plot(df.t.to_numpy(), df.a_z.to_numpy(), ".:", label="z")
    plt.xlabel("t")
    plt.legend()

    plt.subplot(2,3,5)
    plt.tight_layout()
    plt.title("gyro")
    plt.gca().set_aspect('auto')
    plt.plot(df.t.to_numpy(), df.gyro_x.to_numpy(), ".:", label="x")
    plt.plot(df.t.to_numpy(), df.gyro_y.to_numpy(), ".:", label="y")
    plt.plot(df.t.to_numpy(), df.gyro_z.to_numpy(), ".:", label="z")
    plt.xlabel("t")
    plt.legend()

    if "roll" in df.columns:
        plt.subplot(2,3,6)
        plt.tight_layout()
        plt.title("orientation")
        plt.gca().set_aspect('auto')
        plt.plot(df.t.to_numpy(), df.yaw.to_numpy(), ".:", label="yaw")
        plt.plot(df.t.to_numpy(), df.pitch.to_numpy(), ".:", label="pitch")
        plt.plot(df.t.to_numpy(), df.roll.to_numpy(), ".:", label="roll")
        plt.xlabel("t")
        plt.legend()    
        plt.legend()



    df.head()
    
    plt.show()

In [20]:
log_files = list(pathlib.Path.cwd().glob("sphero_log_*.csv"))
log_files.sort(reverse=True)
interact(show_analysis,file_name=[f.name for f in log_files])
pass

interactive(children=(Dropdown(description='file_name', options=('sphero_log_2022-05-29T20:16:26.617.csv', 'sp…

In [21]:
def show_plotly(file_name):
    global df
    df = pd.read_csv(file_name)
    fig1 = px.scatter(df, x="t", y=["roll","pitch","yaw"], title='orientation')
    fig1.show()
    fig2 = px.scatter(df, x="t", y=["a_x","a_y","a_z"], title='acceleration')
    fig2.show()
    fig3 = px.scatter(df, x="t", y=["loc_x","loc_y"], title='location')
    fig3.show


In [22]:
# change style so dropdown is big enough for file paths
from IPython.core.display import HTML
HTML("""
<style>
:root {
--jp-widgets-inline-width: auto
}
</style>
""")

In [23]:
log_files = list(pathlib.Path.cwd().glob("sphero_log_*.csv"))
log_files.sort(reverse=True)
interact(show_plotly,file_name=[f.name for f in log_files])
pass

interactive(children=(Dropdown(description='file_name', options=('sphero_log_2022-05-29T20:16:26.617.csv', 'sp…

In [24]:
def x_rotation(vector,theta):
    """Rotates 3-D vector around x-axis"""
    R = np.array([[1,0,0],[0,np.cos(theta),-np.sin(theta)],[0, np.sin(theta), np.cos(theta)]])
    return np.dot(R,vector)

def y_rotation(vector,theta):
    """Rotates 3-D vector around y-axis"""
    R = np.array([[np.cos(theta),0,np.sin(theta)],[0,1,0],[-np.sin(theta), 0, np.cos(theta)]])
    return np.dot(R,vector)

def z_rotation(vector,theta):
    """Rotates 3-D vector around z-axis"""
    R = np.array([[np.cos(theta), -np.sin(theta),0],[np.sin(theta), np.cos(theta),0],[0,0,1]])
    return np.dot(R,vector)

In [25]:
g = np.array([0,0,1])
x_rotation(g,math.pi/2)

array([ 0.000000e+00, -1.000000e+00,  6.123234e-17])

In [26]:
# pitch only version
rotated_a = [x_rotation(np.array([row.a_x, row.a_y, row.a_z]), -row.pitch*math.pi/180) for row in df.itertuples()]

In [27]:
# roll, pitch only version
rotated_a = [y_rotation(x_rotation(np.array([row.a_x, row.a_y, row.a_z]), -row.pitch*math.pi/180),-row.roll*math.pi/180) for row in df.itertuples()]

In [28]:
rotated_a_x = [r[0] for r in rotated_a]
rotated_a_y = [r[1] for r in rotated_a]
rotated_a_z = [r[2] for r in rotated_a]

In [29]:
df["rotated_a_x"] = rotated_a_x
df["rotated_a_y"] = rotated_a_y
df["rotated_a_z"] = rotated_a_z
df["lateral_g"] = np.hypot(df["rotated_a_x"].to_numpy(),df["rotated_a_y"].to_numpy() )

In [30]:
fig = px.line(df, x="t", y=["a_x", "a_y", "a_z"])
fig.show()

#fig = px.line(df, x="t", y=["rotated_a_x", "rotated_a_y", "rotated_a_z"])
#fig.show()
fig = px.line(df, x="t", y=["world_frame_a_x", "world_frame_a_y", "world_frame_a_z"])
fig.show()

fig = px.line(df, x="t", y=["loc_y","loc_x"])
fig.show()



In [31]:
px.line(df,x="t",y="a_y")

In [32]:
px.line(df,x="t",y="world_frame_a_y")

In [33]:
px.line(df,"t",["v_x", "v_y"])

In [34]:
px.line(df,x="t",y="lateral_g")